In [1]:
import os
from glob import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from dotenv import load_dotenv
load_dotenv()

import textwrap
from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Initialize variables
documents = []
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Define the directory containing the PDF files
pdf_directory = './data'

논문을 벡터 db에 넣기

In [2]:
# pdf를 사용해서 pdf(논문)을 모두 로드
pdf_files = glob(os.path.join(pdf_directory, '*.pdf'))

# Load all PDF files using PyPDFLoader
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    pdf_documents = loader.load()
    documents.extend(pdf_documents)
    
# 텍스트는 RecursiveCharacterTextSplitter를 사용하여 분할
chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = chunk_splitter.split_documents(documents)

# embeddings은 OpenAI의 임베딩을 사용
# vectordb는 chromadb사용함

embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
vectordb = Chroma.from_documents(documents=chunks, embedding=embeddings)
retriever = vectordb.as_retriever()

### 인적정보 가져오기

### 프롬프트

In [3]:
SYS_PROMPT = """
    너는 사회복지사의 업무를 도와주기 위한 챗봇이다. \\
    사회복지 업무와 관련된 메뉴얼과 가이드북을 읽어서 사용자의 질문에 답변할 수 있도록 학습되었다. \\
    너는 주어진 업무를 아주 잘 한다. \\
            """

모델 선언

In [4]:
# 필요한 라이브러리 및 모듈을 임포트합니다.
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 템플릿을 정의합니다.
# SYS_PROMPT는 시스템 메시지로, 템플릿에 포함됩니다. 
# {context}와 {question}은 실행 시 동적으로 채워질 자리표시자입니다.
template = SYS_PROMPT + '''Answer the question based only on the following context:
{context}

Question: {question}

'''

# ChatPromptTemplate.from_template() 메서드를 사용하여 프롬프트 템플릿을 생성합니다.
prompt = ChatPromptTemplate.from_template(template)

# ChatOpenAI 인스턴스를 생성하여 LLM (대규모 언어 모델)을 설정합니다.
# 여기서는 'gpt-4o' 모델을 사용하고, temperature는 0으로 설정하여 출력의 일관성을 높입니다.
model = ChatOpenAI(api_key=OPENAI_API_KEY,model='gpt-4o', temperature=0)

# 문서들을 형식화하는 함수를 정의합니다.
# 각 문서의 페이지 내용을 합쳐 하나의 문자열로 반환합니다.
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# RAG (Retrieval-Augmented Generation) 체인을 연결합니다.
# 이 체인은 문서 검색, 형식화, 프롬프트 적용, 모델 호출, 출력 파싱의 과정을 거칩니다.
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}  # 'context'는 retriever와 format_docs를 통해 설정되고, 'question'은 그대로 전달됩니다.
    | prompt  # 프롬프트 템플릿을 적용합니다.
    | model  # 모델을 호출합니다.
    | StrOutputParser()  # 출력 파서를 통해 모델의 출력을 문자열로 변환합니다.
)

# 체인을 실행합니다.
# 입력 메시지는 질문과 답변 형식의 텍스트입니다.
input_message =  """
    사회복지시설에서의 업무를 수행하다가 발생한 사고들에 대해 어떻게 대처해야 할까요?
"""    # 추가적인 입력 프롬프트가 이어집니다.

# to_markdown() 함수를 호출하여 체인의 결과를 마크다운 형식으로 변환합니다.
to_markdown(rag_chain.invoke(input_message))


> 사회복지시설에서 발생한 사고에 대해 대처하는 방법은 다음과 같습니다:
> 
> 1. **대피 시 행동요령**:
>    - 엘리베이터 사용은 원칙적으로 피하며, 특별한 경우 외에는 사용하지 않도록 합니다.
>    - 시설 관리자의 인원이 부족한 경우, 지역의 협력을 이용하여 신속하게 피난할 수 있도록 조치합니다.
>    - 부상자가 발생한 경우, 응급처치를 실시하고 상태에 따라 응급의료 및 소방서에 연락을 취합니다.
> 
> 2. **시설 유형별 대피 조치사항**:
>    - **노인복지시설**:
>      - 이용자는 대부분 이동 시 많은 시간이 걸리고 직원의 도움이 필요하기 때문에 초기 대피 상황을 신속하게 판단하여 대응해야 합니다.
>      - 대피 유도원 및 노인 지원인력이 부족할 경우, 주변 지역의 봉사자 등을 신속하게 협조받아 신속하고 빠르게 대피시킵니다.
>    - **의료시설**:
>      - 중증환자에서 경상환자까지 다양한 환자가 있기 때문에 병의 특성에 맞게 대응합니다.
>      - 특히 인공투석환자, 난치병 등의 만성질환 환자에 대해서는 각별하게 주의를 두고 특성을 고려하여 대응합니다.
> 
> 3. **대피 후 행동**:
>    - **대피인원 확인 및 점검**:
>      - 대피시설로 모든 인원이 무사하게 이동되었는지를 확인하고 점검을 실시합니다.
>      - 대피시설로 이동이 완료되었을 경우, 이용자의 건강 및 상태를 확인합니다.
>      - 이용자의 건강상태에 변화가 있을 경우, 의사, 간호사 등의 도움을 받거나 근처 병원으로 이동할 수 있도록 조치를 실시합니다.
>    - **2차 재난 발생에 따른 예방조치**:
>      - 지진이나 진진해일, 태풍 등으로 인해 발생되는 2차 재난인 홍수, 산사태 등으로 발생되는 위험에 대해 사전예방을 실시합니다.
>      - 시설 주변 시설물에 대해 안전점검을 위한 순찰을 실시합니다.
> 
> 4. **황사 발생 시 대응활동**:
>    - 황사 발생 상황에 대한 정보를 인지하여 전파하고, 지속적으로 상황 및 정보를 수집합니다.
>    - 시설 이용자에게 재난상황에 대해 정확하게 인지토록 합니다.
>    - 황사로 인한 시설 내 인명피해 현황을 보고하고, 신속한 초동 조치를 실시합니다.
>    - 시설 내 자체 대응활동을 실시하고, 황사발생 조치사항으로 창문 닫기, 마스크 쓰기, 물 마시기 등을 지도합니다.
> 
> 이와 같은 절차를 통해 사회복지시설에서 발생한 사고에 신속하고 적절하게 대처할 수 있습니다.

In [ ]:
class chaingpt:
    def __init__(self,api_key,retriever, sys_prompt="",model="gpt-4o"):
        self.template = sys_prompt + '''Answer the question based only on the following context:
        {context}

        Question: {question}
        '''
        self.prompt = ChatPromptTemplate.from_template(self.template)
        self.model = ChatOpenAI(api_key=api_key,model=model, temperature=1)
        self.chainmodel = (
        {'context': retriever | format_docs, 'question': RunnablePassthrough()}  # 'context'는 retriever와 format_docs를 통해 설정되고, 'question'은 그대로 전달됩니다.
        | self.prompt  # 프롬프트 템플릿을 적용합니다.
        | self.model  # 모델을 호출합니다.
        | StrOutputParser()  # 출력 파서를 통해 모델의 출력을 문자열로 변환합니다.
        )
    def invoke(self,input_message):
        return self.chainmodel.invoke(input_message)
    
#ex
api_key = OPENAI_API_KEY
retriever = vectordb.as_retriever()
sys_prompt = """사용자의 외로움을 판단하고, 사용자에게 적절한 대화 상대가 되어주기 위한 프롬프트를 출력해주세요. """
gpt = chaingpt(api_key,retriever,sys_prompt)
input_message =  """사용자의 외로움은 뭔가요? 적절한 대화상대가 되어주세요."""
print(gpt.invoke(input_message))
        